In [19]:
import tools as tools
import geopandas as gpd
from shapely.geometry import Point
from shapely.geometry.polygon import Polygon

# Standard imports 
import numpy as np
import pandas as pd


# OS and time packages 
import os
import time
import tqdm
import concurrent.futures
from pathlib import Path
from selenium.webdriver.support.ui import WebDriverWait
from selenium.webdriver.support import expected_conditions as EC

# HTML and text processing 
import requests
from bs4 import BeautifulSoup
import json
import re

import time
from bs4 import BeautifulSoup
from selenium import webdriver
from webdriver_manager.chrome import ChromeDriverManager

# Plotting 
import matplotlib.pyplot as plt 
import seaborn as sns

plt.style.use('seaborn-whitegrid')
%matplotlib inline

plt.rc('font', size=14)             # controls default text sizes
plt.rc('axes', titlesize=18)        # fontsize of the axes title
plt.rc('axes', labelsize=18)        # fontsize of the x and y labels
plt.rc('xtick', labelsize=14)       # fontsize of the tick labels
plt.rc('ytick', labelsize=14)       # fontsize of the tick labels
plt.rc('legend', fontsize=14)       # legend fontsize
plt.rc('figure', titlesize=20)      # fontsize of the figure title

plt.rcParams['figure.figsize'] = 10, 4 # set default size of plots

# Filter warnings 
pd.options.mode.chained_assignment = None
from warnings import simplefilter
from sklearn.exceptions import ConvergenceWarning
simplefilter("ignore", category=ConvergenceWarning)

C:\Users\JBH\AppData\Local\Temp\ipykernel_8664\1533326720.py:35: MatplotlibDeprecationWarning: The seaborn styles shipped by Matplotlib are deprecated since 3.6, as they no longer correspond to the styles shipped by seaborn. However, they will remain available as 'seaborn-v0_8-<style>'. Alternatively, directly use the seaborn API instead.
  plt.style.use('seaborn-whitegrid')


Header (state non-commercial/academic intentions)

In [2]:
header = {'name' : 'Jørgen Baun Høst',          'email' : 'pjz633@econ.ku.dk',
          'intention': 'Scrape Boliga for academic purposes'}

Meta data

In [3]:
url = 'https://api.boliga.dk/api/v2/sold/search/results?pageSize=2000&page=1&salesDateMin=1996&salesDateMax=2012&propertytype=1&saleType=1&sort=date-d&buildYearMax=2005'
bbr_test_url = 'https://api.boliga.dk/api/v2/bbrinfo/bbr?id=69cd6d3d-e858-43aa-b530-bd20f132e3b8'
output=tools.get_json(url=url, header=header)
output['meta']

{'pageIndex': 1,
 'pageSize': 2000,
 'totalCount': 790444,
 'totalPages': 396,
 'minPage': 1,
 'maxPage': 6,
 'countFrom': 1,
 'countTo': 2000}

In [4]:
output['results']

[{'estateId': 0,
  'address': 'Ørnhøjvej 7',
  'zipCode': 9240,
  'price': 650000,
  'soldDate': '2012-12-30T23:00:00.000Z',
  'propertyType': 1,
  'saleType': 'Alm. Salg',
  'sqmPrice': 2559.0552,
  'rooms': 8.0,
  'size': 254,
  'buildYear': 1897,
  'change': 0.0,
  'guid': '4F53331C-22C9-4205-9461-27EA1D70E5D9',
  'latitude': 56.918274,
  'longitude': 9.601536,
  'municipalityCode': 851,
  'estateCode': 626742,
  'city': 'Nibe',
  'groupKey': None,
  'canGetVR': True,
  'bfEnr': 8772902},
 {'estateId': 0,
  'address': 'Sdr Lourupvej 14A',
  'zipCode': 6690,
  'price': 250276,
  'soldDate': '2012-12-30T23:00:00.000Z',
  'propertyType': 1,
  'saleType': 'Alm. Salg',
  'sqmPrice': 1881.7744,
  'rooms': 4.0,
  'size': 133,
  'buildYear': 1952,
  'change': 0.0,
  'guid': '7C521EA8-E9F5-40AD-A765-8EC127D3B748',
  'latitude': 55.47826,
  'longitude': 8.801456,
  'municipalityCode': 561,
  'estateCode': 308259,
  'city': 'Gørding',
  'groupKey': None,
  'canGetVR': True,
  'bfEnr': 5051843}

Make list of urls

In [5]:
list_of_url = []

total_pages = output['meta']['totalPages']

for page in range(1, total_pages+1):
    url = f'https://api.boliga.dk/api/v2/sold/search/results?pageSize=2000&page={page}&salesDateMin=1996&salesDateMax=2012&propertytype=1&saleType=1&sort=date-d&buildYearMax=2005'
    list_of_url.append(url)

In [6]:
errors = []

def process_url(id_url_pair):
    id_, url = id_url_pair
    try:
        out = tools.get_json(url, header)
        return id_, out['results']
    except:
        print(f'Error encountered on url {url}')
        errors.append(url)
        pd.DataFrame(errors).to_csv
        return id_, None

id_url_pairs = [(id_, url) for id_, url in enumerate(list_of_url)]

with concurrent.futures.ThreadPoolExecutor() as executor:
    results = list(tqdm.tqdm(executor.map(process_url, id_url_pairs), total=len(id_url_pairs)))

for result in results:
    id_, data = result
    if data is not None:
       with open(f'data/boliga/boliga_{id_}.json', "w") as fp:
        json.dump(data,fp) 

100%|██████████| 396/396 [04:28<00:00,  1.48it/s]


In [9]:
data_dir = Path('data/boliga')
full_df1 = pd.concat(
    pd.DataFrame(json.load(open(json_file)))
    for json_file in data_dir.glob('*.json')
)

In [10]:
df=full_df1.reset_index(drop=True)
df['soldDate']=pd.to_datetime(df['soldDate'])
df['year']=df.soldDate.dt.year
df['month']=df.soldDate.dt.month
df['week']=df.soldDate.dt.weekday
df['time_q']=pd.PeriodIndex(df['soldDate'], freq='Q')

In [11]:
df['time_q']

0         2012Q4
1         2012Q4
2         2012Q4
3         2012Q4
4         2012Q4
           ...  
790439    2006Q4
790440    2006Q4
790441    2006Q4
790442    2006Q4
790443    2006Q4
Name: time_q, Length: 790444, dtype: period[Q-DEC]

In [39]:
df=df.drop(columns=['change'])

In [40]:
df.to_parquet('data/boliga.pq')

In [45]:
gdf = gpd.GeoDataFrame(
    df, geometry=gpd.points_from_xy(df.longitude, df.latitude))
gdf.head()

,estateId,address,zipCode,price,soldDate,propertyType,saleType,sqmPrice,rooms,size,...,estateCode,city,groupKey,canGetVR,bfEnr,year,month,week,time_q,geometry
0,0,Ørnhøjvej 7,9240,650000,2012-12-30 23:00:00+00:00,1,Alm. Salg,2559.0552,8.0,254,...,626742,Nibe,NaN,True,8772902,2012,12,6,2012Q4,POINT (9.60154 56.91827)
1,0,Sdr Lourupvej 14A,6690,250276,2012-12-30 23:00:00+00:00,1,Alm. Salg,1881.7744,4.0,133,...,308259,Gørding,NaN,True,5051843,2012,12,6,2012Q4,POINT (8.80146 55.47826)
2,0,"Rådhusgade 17A, 1. tv",9670,1150000,2012-12-30 23:00:00+00:00,1,Alm. Salg,18253.9690,5.0,63,...,9422,Løgstør,NaN,True,5557002,2012,12,6,2012Q4,POINT (9.25550 56.96835)
3,0,Hennetvedvej 15,5900,1600000,2012-12-30 23:00:00+00:00,1,Alm. Salg,5839.4160,11.0,274,...,4240,Rudkøbing,NaN,True,3075248,2012,12,6,2012Q4,POINT (10.73517 54.85458)
4,782475,Halkærvej 15A,9240,750000,2012-12-30 23:00:00+00:00,1,Alm. Salg,7653.0610,3.0,98,...,626199,Nibe,NaN,True,3292315,2012,12,6,2012Q4,POINT (9.61028 56.95044)


In [47]:
kommune_2006 = gpd.read_file('data/kommune_2006.shp')
kommune_2006

,objectid,navn,enhedid,fra,til,enhedtype,art,SHAPE_Leng,SHAPE_Area,geometry
0,24842.0,Københavns Kommune,120727.0,2002-12-31,2006-12-31,64.0,Kommune,154225.005507,9.028113e+07,"MULTIPOLYGON Z (((12.73424 55.70328 0.00000, 1..."
1,24843.0,Frederiksberg Kommune,120728.0,1970-04-01,2006-12-31,64.0,Kommune,13520.324243,8.709197e+06,"POLYGON Z ((12.53736 55.69661 0.00000, 12.5370..."
2,46149.0,Birkerød Kommune,120748.0,1974-01-04,2006-12-31,64.0,Kommune,35777.227097,3.366869e+07,"POLYGON Z ((12.45040 55.81888 0.00000, 12.4473..."
3,46165.0,Græsted-Gilleleje Kommune,120753.0,1972-10-01,2006-12-31,64.0,Kommune,67562.933801,1.341441e+08,"POLYGON Z ((12.39490 56.03226 0.00000, 12.3952..."
4,46190.0,Jægerspris Kommune,120759.0,1975-01-01,2006-12-31,64.0,Kommune,84485.880031,9.514108e+07,"MULTIPOLYGON Z (((11.92257 55.90000 0.00000, 1..."
...,...,...,...,...,...,...,...,...,...,...
266,46773.0,Thyborøn-Harboøre Kommune,120920.0,2002-12-31,2006-12-31,64.0,Kommune,95733.805758,3.953641e+07,"MULTIPOLYGON Z (((8.18948 56.59492 0.00000, 8...."
267,46789.0,Vinderup Kommune,120925.0,2002-12-31,2006-12-31,64.0,Kommune,109305.320681,2.175008e+08,"MULTIPOLYGON Z (((8.85937 56.39892 0.00000, 8...."
268,46793.0,Åskov Kommune,120926.0,2001-12-31,2006-12-31,64.0,Kommune,77105.236905,2.388402e+08,"POLYGON Z ((8.86526 55.89540 0.00000, 8.86505 ..."
269,46804.0,Galten Kommune,120928.0,2001-12-31,2006-12-31,64.0,Kommune,54492.128242,7.268604e+07,"POLYGON Z ((9.95809 56.13030 0.00000, 9.96044 ..."


In [88]:
regioner=gpd.read_file('data/regioner.shp')
regioner

,objectid,navn,enhedid,fra,til,enhedtype,art,SHAPE_Leng,SHAPE_Area,geometry
0,105612.0,Region Hovedstaden,118711.0,2007-01-01,9999-12-31,66.0,Region,7.371152e+05,2.563160e+09,"MULTIPOLYGON Z (((15.19733 55.31846 0.00000, 1..."
1,105615.0,Region Sjælland,118712.0,2011-11-27,9999-12-31,66.0,Region,2.196684e+06,7.216493e+09,"MULTIPOLYGON Z (((12.54857 54.95316 0.00000, 1..."
2,105616.0,Region Syddanmark,118713.0,2007-01-01,9999-12-31,66.0,Region,2.436420e+06,1.209603e+10,"MULTIPOLYGON Z (((10.68129 54.72772 0.00000, 1..."
3,105620.0,Region Midtjylland,118715.0,2011-01-01,9999-12-31,66.0,Region,2.564054e+06,1.299856e+10,"MULTIPOLYGON Z (((11.67150 56.73861 0.00000, 1..."
4,105617.0,Region Nordjylland,118714.0,2007-01-01,9999-12-31,66.0,Region,1.697705e+06,7.870127e+09,"MULTIPOLYGON Z (((10.98412 57.21880 0.00000, 1..."


In [110]:
gdf_merge=gpd.sjoin(gdf, kommune_2006, predicate='within')
cols_to_drop=['index_right',
 'objectid',
 'navn',
 'enhedid',
 'fra',
 'til',
 'enhedtype',
 'art',
 'SHAPE_Leng',
 'SHAPE_Area']
gdf_merge['kommune_old']=gdf_merge['navn']
gdf_merge=gdf_merge.drop(columns=cols_to_drop)
gdf_merge=gpd.sjoin(gdf_merge, regioner, predicate='within')
gdf_merge['region']=gdf_merge['navn']
gdf_merge=gdf_merge.drop(columns=cols_to_drop)
gdf_merge

C:\Users\JBH\AppData\Local\Temp\ipykernel_8664\4171787026.py:1: UserWarning: CRS mismatch between the CRS of left geometries and the CRS of right geometries.
Use `to_crs()` to reproject one of the input geometries to match the CRS of the other.

Left CRS: None
Right CRS: EPSG:4326

  gdf_merge=gpd.sjoin(gdf, kommune_2006, predicate='within')
C:\Users\JBH\AppData\Local\Temp\ipykernel_8664\4171787026.py:14: UserWarning: CRS mismatch between the CRS of left geometries and the CRS of right geometries.
Use `to_crs()` to reproject one of the input geometries to match the CRS of the other.

Left CRS: None
Right CRS: EPSG:4326

  gdf_merge=gpd.sjoin(gdf_merge, regioner, predicate='within')


,estateId,address,zipCode,price,soldDate,propertyType,saleType,sqmPrice,rooms,size,...,groupKey,canGetVR,bfEnr,year,month,week,time_q,geometry,kommune_old,region
0,0,Ørnhøjvej 7,9240,650000,2012-12-30 23:00:00+00:00,1,Alm. Salg,2559.0552,8.0,254,...,NaN,True,8772902,2012,12,6,2012Q4,POINT (9.60154 56.91827),Nibe Kommune,Region Nordjylland
4,782475,Halkærvej 15A,9240,750000,2012-12-30 23:00:00+00:00,1,Alm. Salg,7653.0610,3.0,98,...,NaN,True,3292315,2012,12,6,2012Q4,POINT (9.61028 56.95044),Nibe Kommune,Region Nordjylland
12,485013,Kirkebjergevej 16,9240,530000,2012-12-30 23:00:00+00:00,1,Alm. Salg,2760.4167,5.0,192,...,NaN,True,3292153,2012,12,6,2012Q4,POINT (9.60734 56.94492),Nibe Kommune,Region Nordjylland
637,798322,Anemonevej 1,9240,1350000,2012-12-17 23:00:00+00:00,1,Alm. Salg,10150.3760,5.0,133,...,NaN,True,5560301,2012,12,0,2012Q4,POINT (9.64762 56.98276),Nibe Kommune,Region Nordjylland
668,0,Halkærvej 54,9240,1925000,2012-12-17 23:00:00+00:00,1,Alm. Salg,21629.2130,4.0,89,...,NaN,True,8772879,2012,12,0,2012Q4,POINT (9.58058 56.92630),Nibe Kommune,Region Nordjylland
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
787897,0,Næblerødvej 14,4250,1450000,2006-12-20 23:00:00+00:00,1,Alm. Salg,11240.3110,5.0,129,...,NaN,True,2520858,2006,12,2,2006Q4,POINT (11.52272 55.34386),Fuglebjerg Kommune,Region Sjælland
789156,0,Kristiansholmsvej 11,4262,1350000,2006-12-11 23:00:00+00:00,1,Alm. Salg,12053.5710,3.0,112,...,NaN,True,2519685,2006,12,0,2006Q4,POINT (11.53148 55.21515),Fuglebjerg Kommune,Region Sjælland
789193,0,Krummerupvej 38,4250,1350000,2006-12-11 23:00:00+00:00,1,Alm. Salg,12053.5710,3.0,112,...,NaN,True,2528554,2006,12,0,2006Q4,POINT (11.51294 55.29964),Fuglebjerg Kommune,Region Sjælland
789940,0,Tystrupvej 3,4250,1805000,2006-12-04 23:00:00+00:00,1,Alm. Salg,18418.3670,4.0,98,...,NaN,True,2529222,2006,12,0,2006Q4,POINT (11.54201 55.37007),Fuglebjerg Kommune,Region Sjælland


In [115]:
house_price_index = pd.read_excel('data/house_price_index.xlsx')
house_price_index['time_q']=pd.PeriodIndex(house_price_index['time_q'], freq='Q')
house_price_index

,region,time_q,house_price_index
0,Region Hovedstaden,1996Q1,0.296
1,Region Hovedstaden,1996Q2,0.304
2,Region Hovedstaden,1996Q3,0.314
3,Region Hovedstaden,1996Q4,0.328
4,Region Hovedstaden,1997Q1,0.325
...,...,...,...
395,Region Nordjylland,2014Q4,1.009
396,Region Nordjylland,2015Q1,1.048
397,Region Nordjylland,2015Q2,1.059
398,Region Nordjylland,2015Q3,1.053


In [122]:
land_taxes = pd.read_excel('data/kmn_grundskyldspromille.xlsx', sheet_name='Sheet2')
land_taxes

,kommune,kommune_amt_2006,kommune_sund_2007,kommune_grundskyld2006,kommune_grundskyld2007,delta_tax,kommune_old
0,Københavns,"32,10","32,00","34,00","34,00",0.00,Københavns Kommune
1,Frederiksberg,"31,20","31,20","27,00","27,00",0.00,Frederiksberg Kommune
2,Ballerup,"32,80","33,58","28,00","28,89",0.89,Ballerup Kommune
3,Brøndby,"32,40","32,47","19,00","19,00",0.00,Brøndby Kommune
4,Dragør,"32,60","32,81","25,70","25,70",0.00,Dragør Kommune
...,...,...,...,...,...,...,...
265,Støvring,"32,60","33,13","22,00","24,14",2.14,Støvring Kommune
266,Sæby,"33,10","32,66","24,00","26,22",2.22,Sæby Kommune
267,Åbybro,"33,00","32,70","22,00","24,82",2.82,Åbybro Kommune
268,Ålborg,"33,40","33,37","27,00","26,95",-0.05,Ålborg Kommune


In [123]:
df_merged = pd.DataFrame(gdf_merge)
df_merged_index = pd.merge(df_merged, house_price_index, left_on=['time_q', 'region'], right_on=['time_q', 'region'])
df_merged_index = pd.merge(df_merged_index, land_taxes, left_on=['kommune_old'], right_on=['kommune_old'])

## Have a look kommune names...

In [124]:
df_merged_index

,estateId,address,zipCode,price,soldDate,propertyType,saleType,sqmPrice,rooms,size,...,geometry,kommune_old,region,house_price_index,kommune,kommune_amt_2006,kommune_sund_2007,kommune_grundskyld2006,kommune_grundskyld2007,delta_tax
0,0,Ørnhøjvej 7,9240,650000,2012-12-30 23:00:00+00:00,1,Alm. Salg,2559.0552,8.0,254,...,POINT (9.60154 56.91827),Nibe Kommune,Region Nordjylland,0.983,Nibe,"33,40","33,37","21,00","26,95",5.95
1,782475,Halkærvej 15A,9240,750000,2012-12-30 23:00:00+00:00,1,Alm. Salg,7653.0610,3.0,98,...,POINT (9.61028 56.95044),Nibe Kommune,Region Nordjylland,0.983,Nibe,"33,40","33,37","21,00","26,95",5.95
2,485013,Kirkebjergevej 16,9240,530000,2012-12-30 23:00:00+00:00,1,Alm. Salg,2760.4167,5.0,192,...,POINT (9.60734 56.94492),Nibe Kommune,Region Nordjylland,0.983,Nibe,"33,40","33,37","21,00","26,95",5.95
3,798322,Anemonevej 1,9240,1350000,2012-12-17 23:00:00+00:00,1,Alm. Salg,10150.3760,5.0,133,...,POINT (9.64762 56.98276),Nibe Kommune,Region Nordjylland,0.983,Nibe,"33,40","33,37","21,00","26,95",5.95
4,0,Halkærvej 54,9240,1925000,2012-12-17 23:00:00+00:00,1,Alm. Salg,21629.2130,4.0,89,...,POINT (9.58058 56.92630),Nibe Kommune,Region Nordjylland,0.983,Nibe,"33,40","33,37","21,00","26,95",5.95
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
757473,0,Nyvej 39,4262,1075000,2007-01-12 23:00:00+00:00,1,Alm. Salg,10141.5100,4.0,106,...,POINT (11.52596 55.26360),Fuglebjerg Kommune,Region Sjælland,1.051,Fuglebjerg,"34,90","32,70","25,00","22,00",-3.00
757474,19967,Nyvej 9,4250,1380000,2007-01-11 23:00:00+00:00,1,Alm. Salg,15333.3330,4.0,90,...,POINT (11.54138 55.30561),Fuglebjerg Kommune,Region Sjælland,1.051,Fuglebjerg,"34,90","32,70","25,00","22,00",-3.00
757475,0,Sorøvej 5,4250,900000,2007-01-09 23:00:00+00:00,1,Alm. Salg,6293.7060,6.0,143,...,POINT (11.54710 55.30669),Fuglebjerg Kommune,Region Sjælland,1.051,Fuglebjerg,"34,90","32,70","25,00","22,00",-3.00
757476,19995,Skælskørvej 112,4261,1275000,2007-01-08 23:00:00+00:00,1,Alm. Salg,10365.8540,3.0,123,...,POINT (11.46541 55.30009),Fuglebjerg Kommune,Region Sjælland,1.051,Fuglebjerg,"34,90","32,70","25,00","22,00",-3.00


In [128]:
after=list(df_merged_index['kommune_old'].unique())
before=list(kommune_2006['navn'].unique())

In [129]:
after

['Nibe Kommune',
 'Løgstør Kommune',
 'Ålborg Kommune',
 'Morsø Kommune',
 'Hirtshals Kommune',
 'Brovst Kommune',
 'Pandrup Kommune',
 'Dronninglund Kommune',
 'Farsø Kommune',
 'Års Kommune',
 'Frederikshavn Kommune',
 'Hobro Kommune',
 'Hanstholm Kommune',
 'Skørping Kommune',
 'Mariager Kommune',
 'Thisted Kommune',
 'Åbybro Kommune',
 'Hadsund Kommune',
 'Sejlflod Kommune',
 'Hjørring Kommune',
 'Sæby Kommune',
 'Løkken-Vrå Kommune',
 'Sydthy Kommune',
 'Nørager Kommune',
 'Ålestrup Kommune',
 'Skagen Kommune',
 'Støvring Kommune',
 'Fjerritslev Kommune',
 'Hals Kommune',
 'Brønderslev Kommune',
 'Arden Kommune',
 'Sindal Kommune',
 'Læsø Kommune',
 'Bramming Kommune',
 'Hårby Kommune',
 'Vejen Kommune',
 'Esbjerg Kommune',
 'Ærø Kommune',
 'Odense Kommune',
 'Vojens Kommune',
 'Ullerslev Kommune',
 'Bov Kommune',
 'Varde Kommune',
 'Give Kommune',
 'Nyborg Kommune',
 'Holsted Kommune',
 'Egtved Kommune',
 'Helle Kommune',
 'Svendborg Kommune',
 'Sønderborg Kommune',
 'Assens Komm

In [130]:
with open('data/kommuner_bef.txt', 'w') as fp:
    for item in before:
        # write each item on a new line
        fp.write("%s\n" % item)
    print('Done')

with open('data/kommuner_after.txt', 'w') as fp:
    for item in after:
        # write each item on a new line
        fp.write("%s\n" % item)
    print('Done')

Done
Done
